In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
stopwords = [word for word in stopwords.words('english')] 
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from nltk.stem import PorterStemmer     # for stemming
from nltk.stem import WordNetLemmatizer # for lemmatization
import numpy as np

[nltk_data] Downloading package stopwords to /home/ifti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ifti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ifti/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ifti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## EDA

In [2]:
with open('CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
    
# # print n lines
# n = 10
# for l in lines[:n]:
#     print(l)
    
    
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["1"]) # note that the dictionary indexes are strings, not numbers. 

    


Number of documents = 1460.

18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


In [5]:
with open('CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
    
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["1"]) # note that the dictionary indexes are strings, not numbers. 

Number of queries = 112.

What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?


In [6]:
def mapping(query_id):
    rel_set = {}
    with open('CISI.REL') as f:
        for l in f.readlines():
            qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
            doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
            if qry_id in rel_set:
                rel_set[qry_id].append(doc_id)
            else:
                rel_set[qry_id] = []
                rel_set[qry_id].append(doc_id)
            if qry_id == query_id:
                print(l.strip("\n"))

    # Print something to see the dictionary structure, etc.
    print(f"\nNumber of mappings = {len(rel_set[query_id])}" + ".\n")
    print(rel_set[query_id]) # note that the dictionary indexes are strings, not numbers.
    return qry_set ,rel_set
    
query_id = str(1)
qry_set ,rel_set= mapping(query_id)

     1     28	0	0.000000
     1     35	0	0.000000
     1     38	0	0.000000
     1     42	0	0.000000
     1     43	0	0.000000
     1     52	0	0.000000
     1     65	0	0.000000
     1     76	0	0.000000
     1     86	0	0.000000
     1    150	0	0.000000
     1    189	0	0.000000
     1    192	0	0.000000
     1    193	0	0.000000
     1    195	0	0.000000
     1    215	0	0.000000
     1    269	0	0.000000
     1    291	0	0.000000
     1    320	0	0.000000
     1    429	0	0.000000
     1    465	0	0.000000
     1    466	0	0.000000
     1    482	0	0.000000
     1    483	0	0.000000
     1    510	0	0.000000
     1    524	0	0.000000
     1    541	0	0.000000
     1    576	0	0.000000
     1    582	0	0.000000
     1    589	0	0.000000
     1    603	0	0.000000
     1    650	0	0.000000
     1    680	0	0.000000
     1    711	0	0.000000
     1    722	0	0.000000
     1    726	0	0.000000
     1    783	0	0.000000
     1    813	0	0.000000
     1    820	0	0.000000
     1    868	0	0.000000
     1    869	0	0.000000


In [7]:
# qry_set

In [9]:
# rel_set['28']


In [11]:
doc_set['35']

'Comparisons of Four Types of Lexical Indicators of Content Rath, G.J. Resnick, A. Savage, T.R. An experiment was conducted to determine which of four types of lexical indicators of content could be utilized best by subjects to determine relevant from irrelevant documents and to answer a set of 100 questions.  The results indicate that there were no major differences between the groups using complete text and abstracts to select relevant documents, but the group utilizing the complete text obtained a significantly higher score on the examination. '

## Preprocessing data

In [12]:
def tags(sentence):
    tokens = nltk.word_tokenize(sentence)
    pos_tagged_tokens = nltk.pos_tag(tokens)
    return pos_tagged_tokens


def preprocess(text):
    text = text.lower().strip()
    text = tags(text)
    punc = "''!()-[]{};:'\,<>./?@#$%^&*_~''"
    processed = ""
    stemmer = PorterStemmer()
    
    for word_ in text:
        word = word_[0]
        tag = word_[1]
        
        if word in stopwords:
            continue
        for i in punc:
            word = word.replace(i, '')
        
        word = stemmer.stem(word)      # stemming the words
        processed = processed + word
        processed = processed + " "
    processed = processed.rstrip()
    return processed    

In [13]:
processed_qry_set = {}
processed_doc_set = {}
voc = []

for key, value in qry_set.items():
    processed = preprocess(value)
    processed_qry_set[key] = processed

for key, value in doc_set.items():
    processed = preprocess(value)
    processed_doc_set[key] = processed
    voc.append(processed)

In [147]:
voc[1]

'use made technic librari slater  m report analysi 6300 act use 104 technic librari unit kingdom  librari use one aspect wider pattern inform use  inform transfer librari restrict use document  take account document use outsid librari  still less inform transfer oral person person  librari act channel proport situat inform transfer  take technic inform transfer whole  doubt proport major one  user technic inform  particularli technolog rather scienc  visit librari rare  reli desk collect handbook  current period person contact colleagu peopl organ  even regular librari user also receiv inform way'

In [151]:
processed_doc_set['1']

'18 edit dewey decim classif comaromi  jp present studi histori dewey decim classif  first edit ddc publish 1876  eighteenth edit 1971  futur edit continu appear need  spite ddc s long healthi life  howev  full stori never told  biographi dewey briefli describ system  first attempt provid detail histori work spur growth librarianship countri abroad'

In [152]:
processed_qry_set['1']

'problem concern make descript titl  difficulti involv automat retriev articl approxim titl  usual relev content articl titl'

## Vectorization

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
vec.fit(voc)
= 14

TfidfVectorizer()

In [154]:
doc_vectors = {}

for key, value in processed_doc_set.items():
    vector = vec.transform([value])
    
    doc_vectors[key] = vector

In [155]:
# doc_vectors['1']

In [156]:
qry_set['3']

'What is information science?  Give definitions where possible.'

# Cosine Similarity 

In [173]:
qn = '111'
query = qry_set[qn]
# print(query)
query = preprocess(query)
query = [query]
# print('preprocessed : ',query)

vector = vec.transform(query)

# print(vector.toarray()[0])
# print(value.toarray()[0])

# consine similarity
similar = []
for key, value in doc_vectors.items():
    # print(vector, value)
    product = sum(vector.toarray()[0] * value.toarray()[0])
    similar.append((product, key))
    
similar = sorted(similar, reverse=True)
k =10
if len(rel_set[qn]) < k:
    k = len(rel_set[qn])
similar = similar[: k]
print(similar)

[(0.5628054992165529, '448'), (0.4743560016626262, '422'), (0.40430700772077716, '635'), (0.39152609071057776, '485'), (0.3716336049278343, '566'), (0.3532262501320893, '570')]


# Evaluation 

In [159]:
from math import log2

count = 0
for i in similar:
    if i[1] in rel_set[qn]:
        count += 1
print("Accuracy : ", count/len(similar))


dcg = 0
c = 1
for i in similar:
    r = int(i[1])
    if c >= 2:
        r = r/log2(c)
    dcg += r
    c += 1
print("DCG : ",dcg) 

Accuracy :  0.9
DCG :  3643.1641290251723
